<a href="https://colab.research.google.com/github/celolima/santander-bootcamp/blob/main/SantanderBootCamp2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Santander Dev Week 2023 (ETL com Python)

**Contexto:** Você é um cientista de dados e precisa indicar filmes para cada cliente.

**Condições do Problema:**

1. A planilha csv será gerada para filmes aleatórios e conterá o seguinte formato:
```
  movie_id, name, category, popularity, budget, invoicing
```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você deverá recomendar um filme para cada cliente, aleatoriamente.
4. Uma vez o filme foi recomendado você vai enviar essas informações de volta para a API, atualizando a lista de "filmes" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.



In [4]:
# Utilize sua própria URL se quiser ;)
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

## Preparação

*Geração do arquivo .csv contendo filmes aleatórios*

In [1]:
import csv
import random

# List of movie categories
categories = [
    "action", "adventure", "comedy", "drama", "fantasy",
    "horror", "musicals", "mystery", "romance", "science fiction",
    "sports", "thriller", "Western"
]

# Generate random movie data
movie_data = []
for movie_id in range(1, 301):
    name = f"Movie {movie_id}"
    category = random.choice(categories)
    popularity = round(random.uniform(1, 10), 2)
    budget = random.randint(1000000, 100000000)
    invoicing = random.choice(["domestic", "international"])

    movie_data.append([movie_id, name, category, popularity, budget, invoicing])

# Write the data to a CSV file
csv_file_path = "random_movies.csv"
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["movie_id", "name", "category", "popularity", "budget", "invoicing"])
    writer.writerows(movie_data)

print(f"CSV file '{csv_file_path}' generated successfully.")

CSV file 'random_movies.csv' generated successfully.


## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [14]:
import pandas as pd

# Convert DataFrame to a list of objects
class Movie:
    def __init__(self, movie_id, name, category, popularity, budget, invoicing):
        self.movie_id = movie_id
        self.name = name
        self.category = category
        self.popularity = popularity
        self.budget = budget
        self.invoicing = invoicing

    def __str__(self):
        return f"Movie ID: {self.movie_id}, Name: {self.name}, Category: {self.category}, Popularity: {self.popularity}, Budget: {self.budget}, Invoicing: {self.invoicing}"

df = pd.read_csv(csv_file_path)
movie_objects = []
for index, row in df.iterrows():
    movie_objects.append(Movie(
        movie_id=row['movie_id'],
        name=row['name'],
        category=row['category'],
        popularity=row['popularity'],
        budget=row['budget'],
        invoicing=row['invoicing']
    ))

In [43]:
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in range(1,20) if (user := get_user(id)) is not None]

## **T**ransform

Obtendo as recomendações de filmes

In [ ]:
def get_random_movie_recommendation(user):
  return movie_objects[user['id']]

for user in users:
  movie = get_random_movie_recommendation(user)
  user['movie'] = []
  user['movie'].append({
      "name": movie.name,
      "category": movie.category
  })


## **L**oad

Atualize a lista de "movies" de cada usuário na API com a nova mensagem gerada.

In [46]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  print(user['movie'])
  success = update_user(user)
  print(f"User {user['name']} - {user['id']} updated? {success}!")

[{'name': 'Movie 2', 'category': 'horror'}]
User Devweekerson - 1 updated? False!
[{'name': 'Movie 14', 'category': 'horror'}]
User anguile - 13 updated? True!
[{'name': 'Movie 15', 'category': 'sports'}]
User Maria - 14 updated? True!
[{'name': 'Movie 16', 'category': 'horror'}]
User Pyterson - 15 updated? True!
[{'name': 'Movie 18', 'category': 'musicals'}]
User Arthur - 17 updated? True!
[{'name': 'Movie 19', 'category': 'romance'}]
User Rides - 18 updated? True!
[{'name': 'Movie 20', 'category': 'horror'}]
User Money Trees - 19 updated? True!
